In [1]:
%load_ext autoreload
%autoreload 2

# AutoModel

In [2]:
import segmentation_models_pytorch as smp
model = smp.FPN(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    in_channels=1,                  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
)

In [3]:
model

FPN(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [35]:
from glasses.nn.models.segmentation.fpn import FPN
from glasses.nn.models.segmentation.unet import UNetEncoder
from functools import partial
import torch

fpn = FPN(encoder=partial(UNetEncoder, widths=[64, 128, 256, 512]))
fpn

FPN(
  (encoder): UNetEncoder(
    (stem): Identity()
    (layers): ModuleList(
      (0): DownLayer(
        (block): Sequential(
          (0): Identity()
          (1): UNetBasicBlock(
            (0): ConvBnAct(
              (conv): Conv2dPad(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act): ReLU(inplace=True)
            )
            (1): ConvBnAct(
              (conv): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act): ReLU(inplace=True)
            )
          )
        )
      )
      (1): DownLayer(
        (block): Sequential(
          (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): UNetBasicBlock(
            (0): ConvBnAct(
             

In [33]:
fpn(torch.randn(1,1,224,224)).shape

torch.Size([1, 512, 28, 28])
torch.Size([1, 512, 28, 28])


RuntimeError: The size of tensor a (512) must match the size of tensor b (256) at non-singleton dimension 1

In [ ]:
from glasses.nn.models.AutoModel import AutoModel
from glasses.nn.models.AutoConfig import AutoConfig
import torch

In [ ]:
model = AutoModel.from_name('resnet18')
# notify to the model we want to have the features
model.encoder.features
model(torch.rand((1,3,224,224)))
# get the features :)
features = model.encoder.features
[print(f.shape) for f in features]

In [ ]:
AutoConfig.from_name('unet')

# UNet

In [ ]:
import torch
from glasses.nn.models.classification import *
from glasses.utils.Storage import ForwardModuleStorage
from glasses.nn.models.segmentation.unet import *
from glasses.nn.models.classification.resnet import ResNetEncoder, ResNet, ResNetBottleneckBlock
from glasses.nn.models.classification.efficientnet import *
from glasses.nn.models.classification import *
from torch import nn

In [ ]:
unet = UNet.from_encoder(lambda *args, **kwargs: ResNet.resnet18(*args, **kwargs))

unet

In [ ]:

unet = UNet.from_encoder(partial(AutoModel.from_name, 'resnet18'))
model.summary(input_shape=(1,384,384))

# Benchmark

In [ ]:
import timm
from glasses.nn.models import *
timm.create_model('seresnet50')

In [ ]:
SEResNet.cse_resnet50()

In [ ]:
enc = EfficientNet.efficientnet_b0().encoder
x = torch.randn((1,3,224,224))

In [ ]:
from glasses.nn.models.classification.efficientnet import *
import timm

from transfer_weights import clone_model
from benchmark import benchmark

src = timm.create_model('tf_efficientnet_lite1', pretrained='True')
dst = EfficientNetLite.efficientnet_lite1(mode='same')

In [ ]:
def from_encoder(model_def, *args, **kwargs):
    model = model_def( *args, **kwargs)
    encoder = nn.Identity()
    if isinstance(model, ResNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  *model.encoder.layers,
                              ],
                              features_widths = model.encoder.widths[1:]
                             )
        
    elif isinstance(model, EfficientNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  model.encoder.layers[1],
                                  model.encoder.layers[2],
                                  model.encoder.layers[3],
                              ],
                              features_widths = [model.encoder.widths[0],
                                                 model.encoder.widths[2],
                                                 model.encoder.widths[3],
                                                 model.encoder.widths[4],
                                                ]
                             )
        
    return encoder

In [ ]:
enc = ResNet.resnet18().encoder

In [ ]:
enc.features_widths()

In [ ]:
enc.stages()

In [ ]:
enc.features

enc(x)

enc.features


In [ ]:
m

In [ ]:
backbone = EfficientNet.efficientnet_b0().encoder


encoder = WithFeatures(backbone, 
                              stages = [
                                  backbone.stem[-2], 
                                  backbone.layers[1],
                                  backbone.layers[2],
                                  backbone.layers[3],
                              ],
                              features_widths = [32, 24, 40, 80]
                             )


m = UNet(encoder = lambda *args, **kwargs: encoder, decoder = partial(UNetDecoder, 
                                                                      widths=[1280, 256, 128, 64, 32, 16]))

In [ ]:
enc.features

In [ ]:
name = 'resnet-asdsa'

name.startswith('resnet')

In [ ]:
m.summary((3, 224, 224))

In [ ]:
dst = clone_model(src, dst)

In [ ]:
dst.summary()

In [ ]:
dst

In [ ]:
# import torch

# transform = dst.configs['efficientnet_lite1'].transform

# benchmark(dst.cuda(), transform)

In [ ]:
# benchmark(src.cuda(), transform)

In [ ]:
src = timm.create_model('efficientnet_b0', pretrained='True')
src

In [ ]:
dst.summary()